In [1]:
import json
from sklearn.neighbors import KernelDensity
import numpy as np
import scipy.stats as stats
import math
import matplotlib
import matplotlib.pyplot as plt
from functools import partial
from simulator import Shooter, Simulator
%matplotlib inline

In [2]:
total_points = {
    1: 40,
    2: 40,
    3: 35,
    4: 30,
    5: 25,
    6: 20
}

In [3]:
athletes_data = json.load(open('participants.json'))

In [4]:
def get_sample(func):
    return func()[0][0]

def get_sample2(func):
    return func()

In [5]:
athletes = []
for i,j in athletes_data.items():
    vals = np.array(j['q'])[:,np.newaxis]
    kde  = KernelDensity(bandwidth=3).fit(vals)
    points = partial(get_sample,kde.sample)

    vals = j['f']
    if len(vals)==0:
        final = lambda: 0.5
    elif len(vals)==1:
        frac = vals[0][0]/total_points[vals[0][1]]
        sampler = lambda: frac
        final = partial(get_sample2,sampler)
    else:
        vals = list(map(lambda x: x[0]/total_points[x[1]] ,vals))
        vals = np.array(vals)[:,np.newaxis]
        kde2  = KernelDensity(bandwidth=0.1).fit(vals)
        final = partial(get_sample,kde2.sample)

    athlete = Shooter(points,final,j['lastName']+' '+j['name'],j['nation'])
    athletes.append(athlete)

In [6]:
tournament = Simulator(athletes)

In [7]:
results = {i.name:[] for i in athletes}
nsim = 10000

In [8]:
for i in range(nsim):
    tournament.run()
    res = list(map(lambda x: x.name,tournament.winners))
    tournament.reset()
    for n,name in enumerate(res):
        results[name].append(n)

In [10]:
confidences = {}

for name, places in results.items():
    mean = np.mean(places)
    interval = stats.t.interval(0.95, len(places)-1, loc=mean, scale=stats.sem(places))
    confidences[name]= {
        'mean':mean,
        'interval': interval,
        'gap': mean-interval[0]
    }

In [11]:
sorted_athletes = list(sorted(confidences.items(),key=lambda x: x[1]['mean']))

In [12]:
fix_results = {}
min_rank = math.floor(sorted_athletes[0][1]['mean'])
for n,(name, data) in enumerate(sorted_athletes):
    rank = data['mean']-min_rank + 1
    fix_results[name] = {
        'rank': rank,
        'gap': data['gap'],
        'sorted_rank': n,
        'nation': athletes_data[name]['nation']
    }

In [13]:
json.dump(fix_results,open('results.json','w'),indent=2)

In [14]:
fix_results

{'REITZ Christian': {'rank': 1.6,
  'gap': 2.111346685224926,
  'sorted_rank': 0,
  'nation': 'GER'},
 'LIN Junmin': {'rank': 3.4000000000000004,
  'gap': 3.238038273290729,
  'sorted_rank': 1,
  'nation': 'CHN'},
 'LI Yuehong': {'rank': 4.1,
  'gap': 3.277309813054394,
  'sorted_rank': 2,
  'nation': 'CHN'},
 'BESSAGUET Clement': {'rank': 4.4,
  'gap': 2.459644452804907,
  'sorted_rank': 3,
  'nation': 'RRA'},
 'QUIQUAMPOIX Jean': {'rank': 5.5,
  'gap': 2.7859148935121647,
  'sorted_rank': 4,
  'nation': 'FRA'},
 'SONG Jong-Ho': {'rank': 5.5,
  'gap': 3.32553346285124,
  'sorted_rank': 5,
  'nation': 'KOR'},
 'HAN Daeyoon': {'rank': 6.4,
  'gap': 2.8851743183170644,
  'sorted_rank': 6,
  'nation': 'KOR'},
 'EKIMOV Leonid': {'rank': 7.800000000000001,
  'gap': 3.231006694366071,
  'sorted_rank': 7,
  'nation': 'ROC'},
 'KOROSTYLOV Pavlo': {'rank': 8.9,
  'gap': 3.5440462216275535,
  'sorted_rank': 8,
  'nation': 'UKR'},
 'GEIS Oliver': {'rank': 8.9,
  'gap': 6.055467922247945,
  'sorte

In [15]:
np.mean(np.random.binomial(20, 0.5, 1))

10.0